### Task 1 (20% credit)

Assume you have a data set as below. It contains records of cars with three features: the type of the car (sports(1) or SUV(2)), the color of the car (red(1) or yellow(2)), and the origin of the car (domestic(1) or imported(2)). And the labels for the data are: stolen(1) and not(0). 

#### Questions:

a) Calculate the following sample probabilities:
P(Red|Stolen), P(SUV|Stolen), P(Domestic|Stolen), P(Red|Not Stolen) , P(SUV|Not Stolen), and P(Domestic|Not Stolen)

b) Suggest a classification for a red, domestic SUV - whether it will be stolen or not - using Naive Bayes classifier. 

Please perform all the necessary computations "by hands" rather than using python code.

In [2]:
import numpy as np
import pandas as pd
y=[1,0,1,0,1,0,1,0,0,1]
X=[[1,1,1,2,2,2,2,2,1,1],[1,1,1,1,1,2,2,2,2,1],[1,1,1,1,2,2,2,1,2,2]]
data=[y]+X
data=pd.DataFrame(data).T
data.columns=['Stolen?','Color','Type','Origin']

In [3]:
data

,Stolen?,Color,Type,Origin
0,1,1,1,1
1,0,1,1,1
2,1,1,1,1
3,0,2,1,1
4,1,2,1,2
5,0,2,2,2
6,1,2,2,2
7,0,2,2,1
8,0,1,2,2
9,1,1,1,2


(a)
P(Red|Yes)=3/5=0.6 

P(Red|No)= 2/5=0.4

P(SUV|Yes)=1/5=0.2 

P(SUV|No) =3/5=0.6

P(Domestic|Yes)=2/5=0.4

P(Domestic|No) =3/5=0.6

(b)

P(Yes|Red,SUV,Domestic)~P(Yes) * P(Red | Yes) * P(SUV | Yes) * P(Domestic|Yes)

=0.5*0.6*0.2*0.4=0.008

P(No) * P(Red | No) * P(SUV | No) * P (Domestic | No)

=0.5*0.6 *0.6 *0.6 =0.108

Answer: No

### Task 2 (25% credit)
Consider a following Guassian Naive Bayes problem.
We use eight factors to predict if people have diabetes or not. The variabls are:

y: The label (0 - no diabetes, 1 - diabetes)

t_pre: Number of times pregnant

glu: Plasma glucose concentration a 2 hours in an oral glucose tolerance test

blood_p: Diastolic blood pressure (mm Hg)

triceps: Triceps skin fold thickness (mm)

serum: 2-Hour serum insulin (mu U/ml)

b_m: Body mass index (weight in kg/(height in m)^2)

pedigree_f: Diabetes pedigree function

age: Age (years)
#### ---------------------------------------------------------------------------------------------------------------------

#### Questions:

a)Train the classifier: use the training data to estimate prior probabilities $P(y=b)$ as well as the parameters (mean and standard deviation) of the sample distributions $P(x_i|y=b)$.

b)Perform the classification for the test sample. 

c)Compare your result to y_test and report the classification accuracy.

In [44]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import scipy
from scipy import stats
%pylab inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
import statsmodels.formula.api as smf
import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import random
#import shapefile

Populating the interactive namespace from numpy and matplotlib


In [5]:


data_train=pd.read_csv("https://serv.cusp.nyu.edu/classes/ML_2016_Spring/session4/HW/dia_train.csv") 
y_train=data_train.iloc[:,1] 
X_train=data_train.iloc[:,2:] 

data_test=pd.read_csv("https://serv.cusp.nyu.edu/classes/ML_2016_Spring/session4/HW/dia_test.csv")
y_test=data_test.iloc[:,1]
X_test=data_test.iloc[:,2:]

In [8]:
def trainNaiveBayes(trainData):
  #training Gausian Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  ind1=tY==0 #Need to change to 0, since my classes are 0 and 1
  ind2=tY==1
  dp=pd.DataFrame(columns=trainData.columns, index=['mu1','sigma1','mu2','sigma2'])
  dp[trainData.columns[0]]['mu1']=1.0*sum(ind1)/len(trainData.index)
  dp[trainData.columns[0]]['mu2']=1.0*sum(ind2)/len(trainData.index)
  for i in trainData.columns[1:]:
    dp.loc['mu1',i]=(trainData[i][ind1]).mean()
    dp.loc['sigma1',i]=(trainData[i][ind1]).std()
    dp.loc['mu2',i]=(trainData[i][ind2]).mean()
    dp.loc['sigma2',i]=(trainData[i][ind2]).std()
  return dp

In [10]:
#Modifided for the zero sigmas !!(This should lead big difference if sample is small!)
def classifyNaiveBayes(classData,dp):
  #classifying using trained Gausian Naive Bayes Classifier
  Y=classData.loc[:,classData.columns[0]]*0
  for j in classData.index:
    P1=dp[classData.columns[0]]['mu1'];
    P2=dp[classData.columns[0]]['mu1'];
    for i in classData.columns[1:]:
        if dp[i]['sigma1']==0:
            #when our sample is very small, how to set this sigma?? if all zeros??
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=0.5) # set 0 to 0.5(maybe need to consider later)
        else:
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=dp[i]['sigma1'])
        
        if dp[i]['sigma2']==0:
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=0.5)
        else:
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=dp[i]['sigma2']) 
    Y[j]=int(P2>P1)
 

  return Y

In [13]:
#(a)
traindata=pd.concat([y_train,X_train],axis=1)
dp=trainNaiveBayes(traindata)
dp

#P(y=0)=0.699; P(y=1)=0.300; mu and sigma are showed in this table.

,y,t_pre,glu,blood_p,triceps,serum,b_m,pedigree_f,age
mu1,0.699153,2.69091,111.467,69.2061,27.2,127.006,31.7091,0.468685,28.3939
sigma1,NaN,2.61006,24.6919,11.7133,10.4369,91.4861,6.33761,0.29175,8.53736
mu2,0.300847,4.07042,144.141,74.5634,33.4789,209.211,35.2239,0.639042,35.7887
sigma2,NaN,3.51862,30.6265,13.7993,9.7627,126.921,6.25849,0.439042,10.2635


In [18]:
#(b) The predicted labels are saved in C. Let's have a look.
from scipy import stats

testdata=pd.concat([y_test,X_test],axis=1)
C=classifyNaiveBayes(testdata,dp)
Result_OS=pd.DataFrame([pd.Series.tolist(y_test),pd.Series.tolist(C)]).T

Result_OS.columns=['real','predicted']
Result_OS.head()

,real,predicted
0,0,0
1,1,1
2,1,1
3,1,1
4,0,0


In [24]:
#c)
acc=Result_OS.loc[:,'real']==Result_OS.loc[:,'predicted']
print("we guessed(OS) correctely by ratio:{0}".format(1.0*mistake.sum()/len(mistake)))

we guessed(OS) correctely by ratio:0.76582278481


### Task 3 (Credit 25%)
We have an artificial data set split, while the training set contains both - labeled (Label_train) and unlabeled (Unlabel) data. Column 'y' is the label, and columns '0','1','2' are categorical variables.

#### Questions:

a) Use the labeled part data_train to predict the labels of X_Label_test, and report the classification accuracy.

b) Improve the classification by using the unlabeled data data_Unlabel and the EM algorithm to predict labels of X_Label_test, and report the new accuracy by EM semi-supervised algorithm (use the same convergence criteria as in the lecture notebook). 

In [25]:
data_train=pd.read_csv("https://serv.cusp.nyu.edu/classes/ML_2016_Spring/session4/HW/EM_train.csv")
y_Label_train=data_train.iloc[:,1] 
X_Label_train=data_train.iloc[:,2:] 

data_test=pd.read_csv("https://serv.cusp.nyu.edu/classes/ML_2016_Spring/session4/HW/EM_test.csv")
y_Label_test=data_test.iloc[:,1]
X_Label_test=data_test.iloc[:,2:]

data_Unlabel=pd.read_csv("https://serv.cusp.nyu.edu/classes/ML_2016_Spring/session4/HW/EM_Unlabel.csv")
X_Unlabel=data_Unlabel.iloc[:,1:]

Part (a)

In [26]:

def trainNaiveBayesDiscrete(trainData):
  #training discrete Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  m=max([trainData[j][i] for j in trainData.columns[1:] for i in trainData.index]) #maximal number of classes in each feature of a training set
  #create output data structure for the probabilities
  dp=[pd.DataFrame(columns=trainData.columns, index=range(1,m+1)), pd.DataFrame(columns=trainData.columns, index=range(1,m+1))]
  #split the training data between two labels
  ind1=tY==0
  ind2=tY==1
  #estimate P(y=b)  
  dp[0][trainData.columns[0]][1]=1.0*ind1.sum()/len(trainData.index)
  dp[1][trainData.columns[0]][1]=1.0*ind2.sum()/len(trainData.index)
  #estimate conditional probabilities P(x|y=b)
  for j in trainData.columns[1:]:
    for i in range(1,m+1):
        dp[0].loc[i,j]=1.0*(trainData[j][ind1]==i).sum()/ind1.sum();
        dp[1].loc[i,j]=1.0*(trainData[j][ind2]==i).sum()/ind2.sum();
  return dp


def classifyNaiveBayesDiscrete(classData,dp):
  #classifying using trained discrete Naive Bayes Classifier
  Y=classData[classData.columns[0]]*0
  for i in classData.index:
    P1=dp[0][classData.columns[0]][1];
    P2=dp[1][classData.columns[0]][1];
    for j in classData.columns[1:]:
      P1=P1*dp[0][j][classData[j][i]]
      P2=P2*dp[1][j][classData[j][i]]
    Y[i]=int(P2>P1) 
  return Y

In [34]:
# get the probability distribution firstly
trainData=pd.concat([y_Label_train,X_Label_train],axis=1)
dp=trainNaiveBayesDiscrete(trainData)
print dp[0]
print dp[1]

          y    0    1    2
1  0.555556    0  0.2    0
2       NaN  0.2    0  0.2
3       NaN  0.2    0    0
4       NaN  0.2  0.2  0.2
5       NaN    0    0    0
6       NaN  0.4  0.6  0.6
          y     0     1     2
1  0.444444   0.5  0.25  0.25
2       NaN  0.25   0.5   0.5
3       NaN     0  0.25     0
4       NaN     0     0     0
5       NaN  0.25     0     0
6       NaN     0     0  0.25


In [30]:
#Calculate the predicted labels:
testdata=pd.concat([y_Label_test,X_Label_test],axis=1)
C=classifyNaiveBayesDiscrete(testdata,dp)

In [31]:
# Report the accuracy of prediction
acc=format(100.0*sum(C==y_Label_test)/len(y_Label_test))
print ("We correctly classified {0} percents of the trips based on the labeled data only".format(acc))   

We correctly classified 63.8888888889 percents of the trips based on the labeled data only


Part (b)

In [45]:
#implementation of Expectation-Maximization algorithm for partially labeled data
def EM(X_Label,y_Label,X_Unlabel,dp):
  t = 0  
  haslabels=len(y_Label)>0

  while True:
    t = t + 1

    classData=X_Unlabel
    # Now we want to calculate P(y=1|x) and P(y=2|x) for all observations xj. (these are bunch of scalars)
    # we need this to calculate new dp. Basically speaking, for every new iteration we need a new dp.

    #for y=1 and y=2

    p_x_1=[] #unnormalized P(y=1|x)
    p_x_2=[] #unnormalized P(y=2|x)
    cols=dp[0].columns

    for i in classData.index:
        P1=dp[0][cols[0]][1];
        P2=dp[1][cols[0]][1];
        for j in classData.columns:
            P1=P1*dp[0][j][classData[j][i]]
            P2=P2*dp[1][j][classData[j][i]]
        p_x_1.append(P1)
        p_x_2.append(P2)

    #Rescale p_x_1 and p_x_2:
    summ=np.asarray(p_x_1)+np.asarray(p_x_2)
    p_x_1_s=np.asarray(p_x_1)/summ
    p_x_2_s=np.asarray(p_x_2)/summ
    inds_1 = np.where(np.isnan(p_x_1_s))
    inds_2 = np.where(np.isnan(p_x_2_s))
    p_x_1_s[inds_1]=0.5
    p_x_2_s[inds_2]=0.5
    #Now let's calculate P(y=1) and P(y=2)
    p_1=p_x_1_s.sum()/len(p_x_1_s)
    p_2=p_x_2_s.sum()/len(p_x_2_s)


    #Now let's calculate the probability distribution of P(xi|y=1) and P(xi|y=2)
    
    m=max([classData[j][i] for j in classData.columns for i in classData.index]) #maximal number of classes in each feature of a training set

    #create output data structure for the probabilities - new iteration
    
    dp1=[pd.DataFrame(columns=cols, index=range(1,m+1)), pd.DataFrame(columns=cols, index=range(1,m+1))]

    #P(y=b)  
    dp1[0][cols[0]][1]=p_1
    dp1[1][cols[0]][1]=p_2


    #estimate conditional probabilities P(x|y=b) -do we add labeled data to fit?

    temp=np.concatenate((np.asmatrix(X_Unlabel),np.asarray(pd.DataFrame(p_x_1_s)),np.asarray(pd.DataFrame(p_x_2_s))), axis=1)
    temp=pd.DataFrame(temp)
    if haslabels:
        ######!!! y_Label=0,1,...
        temp_l=np.concatenate((np.asmatrix(X_Label),np.asmatrix(1*(y_Label==0)).transpose(),np.asmatrix(1*(y_Label==1)).transpose()),axis=1)
        temp_l=pd.DataFrame(temp_l)
        pd.concat([temp,temp_l])
   

    for j in range(1,len(dp[0].T)):
        for i in range(len(dp[0])):
            ###!!!!not 4,5
            dp1[0].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-2].sum()/temp.iloc[:,-2].sum()
            dp1[1].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-1].sum()/temp.iloc[:,-1].sum()

        ############################################################################################
    # Now we use dp to decide whether to continue our iterations
    
    if (sum(sum((dp1[0]-dp[0])**2))+sum(sum((dp1[1]-dp[1])**2)))<0.001: #if dp does not change much
        break
    else: 
        dp=dp1  #save new dp and perform next iteration

        
    ###############################################################################################
        #Calculate the log-likelihood
        
        L=0
        
        for i in classData.index:
            P1=dp[0][cols[0]][1];
            P2=dp[1][cols[0]][1];
            for j in classData.columns:
                P1=P1*dp[0][j][classData[j][i]]
                P2=P2*dp[1][j][classData[j][i]]
            temp=math.log(P1+P2)
            L=L+temp
        if haslabels: 
            ########
          for i in X_Label.index:
            yi=y_Label[i]
            P=dp[yi][cols[0]][1];
            for j in X_Label.columns:
                P=P*dp[yi][j][X_Label[j][i]]
            L=L+math.log(P)
        
        print ("Iteration {0}: log maximum liklihood = {1}".format(t,L))   
        
        
  return dp



In [47]:
#perform EM estimation for theta. dp is taken from part a.
dpEM=EM(X_Label_train,y_Label_train,X_Unlabel,dp)
#OS test
C=classifyNaiveBayesDiscrete(testdata,dpEM) #classify test data with a new theta given by EM
acc=100.0*sum(C==y_Label_test)/len(y_Label_test)
print ("After EM we correctly classified {0} percents of the trips".format(acc))

Iteration 1: log maximum liklihood = -590.452072842
Iteration 2: log maximum liklihood = -572.346722303
Iteration 3: log maximum liklihood = -559.190769042
Iteration 4: log maximum liklihood = -556.633109435
Iteration 5: log maximum liklihood = -556.067798435
After EM we correctly classified 94.4444444444 percents of the trips


### Task 4 (Credit 30%)
For the similar artifitial data uploaded below:

#### Question: 

a) Apply the EM algorithm (no observed labels, random initial choice of $\theta$) for clustering the data records into two clusters. Report your result (a vector of cluster numbers for each data record). 

b) Repeat the clustering 10 times with different random choices of $\theta$ and analyze the stability of the clustering (matching labels accross different clusterings (use the choice of 0 and 1 labels best matching the previous clustering), estimate average label and its standard error for each record).

In [50]:
data=pd.read_csv("https://serv.cusp.nyu.edu/classes/ML_2016_Spring/session4/HW/EM_Cluster.csv")
X=data.iloc[:,1:]

In [52]:
#(a)
# Generate a random theta
b=np.random.uniform(0,1,len(dp[0]))
b=np.asarray(b)/float(np.asarray(b).sum())
dp[0].iloc[0,0]=np.random.uniform(0,1)
dp[1].iloc[0,0]=dp[1].iloc[0,0]=1-dp[0].iloc[0,0]
for j in range(1,len(dp[0].T)):
    dp[0].iloc[:,j]=b
for j in range(1,len(dp[1].T)):
    b=np.random.uniform(0,1,len(dp[1]))
    b=np.asarray(b)/float(np.asarray(b).sum())
    dp[1].iloc[:,j]=b
#perform EM estimaton for theta
dpEM=EM([],[],X_Unlabel,dp)
#Write down the clustering result in C
C=classifyNaiveBayesDiscrete(testdata,dpEM) 
print C

Iteration 1: log maximum liklihood = -547.708551189
Iteration 2: log maximum liklihood = -543.632550236
Iteration 3: log maximum liklihood = -539.300168666
Iteration 4: log maximum liklihood = -535.227521656
Iteration 5: log maximum liklihood = -532.224841652
Iteration 6: log maximum liklihood = -530.477702801
Iteration 7: log maximum liklihood = -529.30668097
Iteration 8: log maximum liklihood = -528.179309407
Iteration 9: log maximum liklihood = -526.70959117
Iteration 10: log maximum liklihood = -524.452440999
Iteration 11: log maximum liklihood = -521.449758353
Iteration 12: log maximum liklihood = -518.122329914
Iteration 13: log maximum liklihood = -513.845631444
Iteration 14: log maximum liklihood = -509.617359123
Iteration 15: log maximum liklihood = -507.615671708
Iteration 16: log maximum liklihood = -507.094427927
0     0
1     0
2     1
3     0
4     1
5     1
6     1
7     1
8     1
9     1
10    0
11    0
12    1
13    0
14    1
15    1
16    0
17    0
18    1
19    1
20 

In [58]:
# b) Do it 10 times. And compare the clustering result
answer=[]
for i in range(10):
#initialize theta randomly
    b=np.random.uniform(0,1,len(dp[0]))
    b=np.asarray(b)/float(np.asarray(b).sum())
    dp[0].iloc[0,0]=np.random.uniform(0,1)
    dp[1].iloc[0,0]=dp[1].iloc[0,0]=1-dp[0].iloc[0,0]
    for j in range(1,len(dp[0].T)):
        dp[0].iloc[:,j]=b
    for j in range(1,len(dp[1].T)):
        b=np.random.uniform(0,1,len(dp[1]))
        b=np.asarray(b)/float(np.asarray(b).sum())
        dp[1].iloc[:,j]=b
    #perform EM estimaton for theta
    dpEM=EM([],[],X_Unlabel,dp)
    #OS test
    C=classifyNaiveBayesDiscrete(testdata,dpEM) #classify test data with a new theta given by EM
    answer.append(C.values.tolist())
answer=pd.DataFrame(answer).T

Iteration 1: log maximum liklihood = -551.622252386
Iteration 2: log maximum liklihood = -549.572758914
Iteration 3: log maximum liklihood = -547.722997458
Iteration 4: log maximum liklihood = -546.066376714
Iteration 5: log maximum liklihood = -544.699060641
Iteration 6: log maximum liklihood = -543.408365954
Iteration 7: log maximum liklihood = -541.87423782
Iteration 8: log maximum liklihood = -540.017645068
Iteration 9: log maximum liklihood = -537.828806638
Iteration 10: log maximum liklihood = -534.98176845
Iteration 11: log maximum liklihood = -530.502472417
Iteration 12: log maximum liklihood = -523.009794221
Iteration 13: log maximum liklihood = -514.358299491
Iteration 14: log maximum liklihood = -509.073949214
Iteration 15: log maximum liklihood = -506.885051976
Iteration 16: log maximum liklihood = -505.963292913
Iteration 1: log maximum liklihood = -552.5032281
Iteration 2: log maximum liklihood = -548.16277931
Iteration 3: log maximum liklihood = -538.867842774
Iteration 

In [59]:
answer

,0,1,2,3,4,5,6,7,8,9
0,1,1,1,1,1,1,1,1,0,0
1,1,0,1,1,1,1,0,1,0,0
2,0,1,0,0,0,0,1,0,1,1
3,1,0,1,1,1,1,0,1,0,0
4,0,1,0,0,0,0,1,0,1,1
5,0,1,0,0,0,0,1,0,1,1
6,0,1,0,0,0,0,1,0,1,1
7,0,1,0,0,0,0,1,0,1,1
8,0,1,0,0,0,0,1,0,1,1
9,0,1,0,0,0,0,1,0,1,1


In [61]:
# Now Let's compare the results from columns 1 to 9 to the result 0 to see the similarity between them.
# Remember we reach the results everytime from a random Theta.
#(! when we talk about stability of clustering, we want to show the clusters(results) in our 
#experiments are similar. However,we do not care about the labels of clusters, since we do not
#have any labels. So we need to be careful when we compare the results. For example, if we have
#four elements and did two experiments, first time the labels=
#[0,0,1,1] and the second time the labels=[1,1,0,0], We regard the results are same.)
total_accurace=0
for i in range(1,10):
    temp=(answer.iloc[:,i]-answer.iloc[:,i-1]).abs()
    temp=temp.sum()/len(answer)
    if temp<0.5:
        total_accurace=total_accurace+(1-temp)
    else:
        total_accurace=total_accurace+temp

print("the average matching rate between 1-9 times and 0 time, is:{}".format(1.0*total_accurace/9))   

the average matching rate between 1-9 times and 0 time, is:1.0


In [ ]:
# So, our EM cloustering is very stable(by 10 times experient we get same clusters) and we 
#do not have any errors here in my experiments.